In [ ]:
from periphery import Serial
import time

DEV = "/dev/ttyAS0"
BAUD = 9600

KNOTS_TO_KMH = 1.852


def parse_gga(line: str):
    # $GNGGA,time,lat,N,lon,E,fix,numsats,hdop,alt,M,...
    parts = line.split(",")
    if len(parts) < 8:
        return None
    fix_q = parts[6].strip()  # 0=no fix, 1=GPS fix, 2=DGPS, etc.
    sats = parts[7].strip()
    try:
        fix_q_i = int(fix_q) if fix_q else 0
    except ValueError:
        fix_q_i = 0
    try:
        sats_i = int(sats) if sats else 0
    except ValueError:
        sats_i = 0
    return fix_q_i, sats_i


def parse_speed_kmh(line: str):
    # Prefer VTG if it contains km/h
    if line.startswith(("$GNVTG", "$GPVTG")):
        parts = line.split(",")
        # Standard VTG: ..., speed_knots, N, speed_kmh, K*CS
        if len(parts) > 9:
            sp_kmh = parts[7].strip()
            if sp_kmh:
                try:
                    return float(sp_kmh)
                except ValueError:
                    return None

    # RMC: speed over ground in knots (field 7), status field 2 must be 'A'
    if line.startswith(("$GNRMC", "$GPRMC")):
        parts = line.split(",")
        if len(parts) > 7:
            status = parts[2].strip()  # A=valid, V=void
            sog_knots = parts[7].strip()
            if status == "A" and sog_knots:
                try:
                    return float(sog_knots) * KNOTS_TO_KMH
                except ValueError:
                    return None

    return None


def main():
    ser = Serial(DEV, BAUD)
    buf = bytearray()

    fix_q = 0
    sats = 0
    speed_kmh = None
    last_update = 0.0

    print("Reading GPS on", DEV, "@", BAUD)
    print("Waiting for fix... (go outdoors / near a window). Ctrl+C to exit.")

    try:
        while True:
            data = ser.read(256, timeout=1.0)
            if data:
                buf.extend(data)

                while b"\n" in buf:
                    line, _, rest = buf.partition(b"\n")
                    buf = bytearray(rest)

                    s = line.decode("ascii", errors="ignore").strip()
                    if not s.startswith("$"):
                        continue

                    # Ignore noisy TXT spam (optional)
                    if s.startswith(("$GNTXT", "$GPTXT")):
                        continue

                    if s.startswith(("$GNGGA", "$GPGGA")):
                        g = parse_gga(s)
                        if g:
                            fix_q, sats = g

                    sp = parse_speed_kmh(s)
                    if sp is not None:
                        speed_kmh = sp

            # print dashboard line 4x/sec
            now = time.time()
            if now - last_update >= 0.25:
                last_update = now

                fix_str = "NO FIX" if fix_q == 0 else f"FIX({fix_q})"
                if speed_kmh is None or fix_q == 0:
                    sp_str = "--"
                else:
                    sp_str = f"{speed_kmh:6.2f} km/h"

                print(f"{fix_str:8} | sats: {sats:2d} | speed: {sp_str}")

    except KeyboardInterrupt:
        print("\nExited.")
    finally:
        ser.close()


if __name__ == "__main__":
    main()